In [ ]:
# you don't need to run this if you use 'pip install -r requirements'

# !apt install tesseract-ocr
# !apt install libtesseract-dev
# !pip install pillow
# !pip install pytesseract
# !pip install transformers
# !pip install PyPDF2
# !pip install PyMuPDF

In [5]:
import PyPDF2


with open('../ressources/loi_english.pdf', 'rb') as pdf_file:
   
    pdf_reader = PyPDF2.PdfReader(pdf_file)
   
   
    articles = []
   
   
    num_pages = len(pdf_reader.pages)
   
   
    for page in range(5,num_pages):
       
        page_obj = pdf_reader.pages[page]
        page_text = page_obj.extract_text()
       
       
        article_list = page_text.split('Article ')
        article_list.pop(0)  
       
       
        for article in article_list:
            articles.append('Article ' + article)
           
   
    print(articles)

['Article  amended  by  ', 'Article  1  of  Law  No.  24.03  amending  and  supplementing  the  Penal  \nCode,  promulgated  by  Dahir  No.  1-03-207  of  16  Ramadan  1424  (November  11,  2003),  \nOfficial  Bulletin  No.  5178  of  22  kaada  1424  (January  15,  2004),  p.  114.SAFETY\nTITLE  ONE  PENALTIESBOOK  ONE  OF  PENALTIES  AND  MEASURES  OF\nSection  13\nSection  14(Items  13  to  109)\n(Items  14  to  60)Machine Translated by Google', 'Article  1  of  Law  No.  3-80  amending  certain  provisions  of  the  Penal  \nCode  promulgated  by  Dahir  No.  1-81-283  of  11  Rejeb  1402  (May  6,  1982),  Official  Bulletin  No.  \n3636  of  15  Ramadan  1402  (July  7,  1982),  p.351,  then  by  Law  No.  25-93  amending  the  penal  \ncode  promulgated  by  Dahir  No.  1-94-284  of  Safar  15,  1415  (July  25,  1994),  Official  Bulletin  No.  \n°  4266  of  24  safar  1415  (August  3,  1994),  p.  371.\n7  -  Paragraph  modified  by  the  first  article  of  law  n°  3-80  m

In [15]:
import fitz
from transformers import pipeline
from tqdm import tqdm
import random

models_checkpoints = {"BertForQuestionAnswering": "bert-large-uncased-whole-word-masking-finetuned-squad"}

# Extract text from PDF
random.shuffle(articles)
paragraphs = articles

# Use pipeline to answer questions
pipe = pipeline("question-answering", model=models_checkpoints["BertForQuestionAnswering"])
question = "what if i witness a crime and i was blackmailed"

# Find top 2 most probable answers
answers = []
i = 0
for p in tqdm(paragraphs):
    if p:
        answer = pipe(question=question, context=p)
        answer["paragraph"] = p
        answers.append(answer)
        i+=1
    if i>5:
      break
        
top_answers = sorted(answers, key=lambda x: x["score"], reverse=True)[:2]

# Print top 2 answers
for i, a in enumerate(top_answers):
    print(f"Answer {i+1}: {a['answer']}\nParagraph: {a['paragraph']}\nScore: {a['score']}\n")


  3%|▎         | 5/171 [00:16<09:11,  3.32s/it]

Answer 1: law  n°  79-03.Machine Translated by Google
Paragraph: Article  amended  and  supplemented  by  article  2  of  the  aforementioned  law  n°  79-03.Machine Translated by Google
Score: 0.036888960748910904

Answer 2: Article  727 :  The  public  prosecutor  may  require  the  syndic
Paragraph: Article  727 :  The  public  prosecutor  may  require  the  syndic  to  hand  over  all  the  deeds  and  documents  
held  by  him.
Score: 0.035152193158864975



In [14]:
top_answers[1]['paragraph'] in paragraphs

True